In [1]:
import os

In [7]:
%pwd

'c:\\Users\\hazsy\\Desktop\\Haz(Padhai Likhai)\\Cancer_MLFlow\\CANCER_CLASSIFICATION\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\hazsy\\Desktop\\Haz(Padhai Likhai)\\Cancer_MLFlow\\CANCER_CLASSIFICATION'

In [1]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_size: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [2]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [3]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])
        
        prepare_base_model_config=PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
        params_image_size=self.params.IMAGE_SIZE,
        params_learnin_rate=self.params.LEARNING_RATE,
        params_include_top=self.params.INCLUDE_TOP,
        params_weights=self.params.WEIGHTS,
        params_classes=self.params.CLASSES
        )
        
        return prepare_base_model_config

In [5]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
      --------------------------------------- 0.1/5.6 MB 2.3 MB/s eta 0:00:03
     -- ------------------------------------- 0.3/5.6 MB 4.1 MB/s eta 0:00:02
     ------ --------------------------------- 0.9/5.6 MB 6.4 MB/s eta 0:00:01
     --------- ------------------------------ 1.3/5.6 MB 6.6 MB/s eta 0:00:01
     ------------ --------------------------- 1.8/5.6 MB 7.0 MB/s eta 0:00:01
     --------------- ------------------------ 2.1/5.6 MB 7.5 MB/s eta 0:00:01
     ------------------ --------------------- 2.5/5.6 MB 7.3 MB/s eta 0:00:01
     --------------------- ------------------ 3.0/5.6 MB 7.7 MB/s eta 0:00:01
     -------------------------- ------------- 3.7/5.6 MB 8.1 MB/s eta 0:00:01
     ---------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydantic 2.5.3 requires typing-extensions>=4.6.1, but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.14.6 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.


In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [10]:
class PrepareBaseModel:
    def __init__(self,config: PrepareBaseModelConfig):
        self.config= config 

    def get_base_model(self):
        self.model=tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )
        
        self.save_model(path=self.config.base_model_path,model=self.model)
        
    @staticmethod
    def _prepare_full_model(model,classes,freeze_all,freeze_till,learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable=False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable=False
                
        flatten_in=tf.keras.layers.Flatten()(model.output)
        prediction=tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)
        
        full_model = tf.keras.models.Model(
            inputs=model.input,
            output=prediction
            
        )
        
        full_model.compile(
            optimizer=tf.train.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.train.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
        
        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        
        
    
        @staticmethod
        def save_model(path: Path, model: tf.keras.Model):
            model.save(path)